In [1]:
# filling missing values with autoencoder

import pandas as pd
import numpy as np
import tensorflow as tf

# Load your data
df = pd.read_csv('../dataset/metadata_unfilled.csv')

# Define function to convert booleans to strings
def convert_bool_to_str(value):
    if isinstance(value, bool):
        return "True" if value else "False"
    return value

# Apply the conversion function to each column
for col in df.columns:
    df[col] = df[col].map(convert_bool_to_str)

# Columns with floats to convert
float_columns = ['fitspatrick', 'diameter_1', 'diameter_2']

# Iterate over each specified column, convert non-NaN values to integers, and set to Int64 type
for column in float_columns:
    df[column] = df[column].apply(lambda x: int(x) if pd.notna(x) else pd.NA).astype("Int64")

# Display unique values in each column
for column in df.columns:
    print(f"Column: {column}")
    print(df[column].unique())
    print("\n")

2024-11-06 12:56:00.595800: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 12:56:00.635537: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 12:56:00.635570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 12:56:00.636470: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 12:56:00.642501: I tensorflow/core/platform/cpu_feature_guar

Column: patient_id
['PAT_1516' 'PAT_46' 'PAT_1545' ... 'PAT_491' 'PAT_1343' 'PAT_1714']


Column: lesion_id
[1765  881 1867 ...  934 3156 3189]


Column: smoke
[nan 'False' 'True']


Column: drink
[nan 'False' 'True']


Column: background_father
[nan 'POMERANIA' 'GERMANY' 'BRAZIL' 'NETHERLANDS' 'ITALY' 'POLAND' 'UNK'
 'PORTUGAL' 'BRASIL' 'CZECH' 'AUSTRIA' 'SPAIN' 'ISRAEL']


Column: background_mother
[nan 'POMERANIA' 'ITALY' 'GERMANY' 'BRAZIL' 'UNK' 'POLAND' 'NORWAY'
 'PORTUGAL' 'NETHERLANDS' 'FRANCE' 'SPAIN']


Column: age
[ 8 55 77 75 79 53 52 74 68 58 45 34  9 78 60 64 61 62 67 71 66 38 54 82
 63 49 76 31 36 17 44 56 26 47 80 48 51 35 87 41 22 69 59 57 65 83 92 33
 46 50 70 85 40 73 10 86 72 14 21 39 94 43 90 89 13 30 81 84 88 37 32 16
 29 24 42 20 23 28 27 91 18 12 25  6]


Column: pesticide
[nan 'False' 'True']


Column: gender
[nan 'FEMALE' 'MALE']


Column: skin_cancer_history
[nan 'True' 'False']


Column: cancer_history
[nan 'True' 'False']


Column: has_piped_water
[nan 'True

In [2]:
print(df.head())

  patient_id  lesion_id  smoke  drink background_father background_mother  \
0   PAT_1516       1765    NaN    NaN               NaN               NaN   
1     PAT_46        881  False  False         POMERANIA         POMERANIA   
2   PAT_1545       1867    NaN    NaN               NaN               NaN   
3   PAT_1989       4061    NaN    NaN               NaN               NaN   
4    PAT_684       1302  False   True         POMERANIA         POMERANIA   

   age pesticide  gender skin_cancer_history  ... diameter_2 diagnostic  \
0    8       NaN     NaN                 NaN  ...       <NA>        NEV   
1   55     False  FEMALE                True  ...          5        BCC   
2   77       NaN     NaN                 NaN  ...       <NA>        ACK   
3   75       NaN     NaN                 NaN  ...       <NA>        ACK   
4   79     False    MALE                True  ...          5        BCC   

    itch   grew   hurt  changed  bleed elevation                 img_id  \
0  False  F

In [3]:
dfcopy = df.copy()

df = df.drop(columns=['patient_id', 'lesion_id', 'img_id'])
for column in df.columns:
    print(f"Column: {column}")
    print(df[column].unique())
    print("\n")


df_with_na = df[df.isna().any(axis=1)]

print(df_with_na)
df_cleaned = df.dropna()

Column: smoke
[nan 'False' 'True']


Column: drink
[nan 'False' 'True']


Column: background_father
[nan 'POMERANIA' 'GERMANY' 'BRAZIL' 'NETHERLANDS' 'ITALY' 'POLAND' 'UNK'
 'PORTUGAL' 'BRASIL' 'CZECH' 'AUSTRIA' 'SPAIN' 'ISRAEL']


Column: background_mother
[nan 'POMERANIA' 'ITALY' 'GERMANY' 'BRAZIL' 'UNK' 'POLAND' 'NORWAY'
 'PORTUGAL' 'NETHERLANDS' 'FRANCE' 'SPAIN']


Column: age
[ 8 55 77 75 79 53 52 74 68 58 45 34  9 78 60 64 61 62 67 71 66 38 54 82
 63 49 76 31 36 17 44 56 26 47 80 48 51 35 87 41 22 69 59 57 65 83 92 33
 46 50 70 85 40 73 10 86 72 14 21 39 94 43 90 89 13 30 81 84 88 37 32 16
 29 24 42 20 23 28 27 91 18 12 25  6]


Column: pesticide
[nan 'False' 'True']


Column: gender
[nan 'FEMALE' 'MALE']


Column: skin_cancer_history
[nan 'True' 'False']


Column: cancer_history
[nan 'True' 'False']


Column: has_piped_water
[nan 'True' 'False']


Column: has_sewage_system
[nan 'True' 'False']


Column: fitspatrick
<IntegerArray>
[<NA>, 3, 1, 2, 4, 5, 6]
Length: 7, dtype: Int64


In [4]:
df = df_cleaned

for column in df.columns:
    print(f"Column: {column}")
    print(df[column].unique())
    print("\n")

Column: smoke
['False' 'True']


Column: drink
['False' 'True']


Column: background_father
['POMERANIA' 'GERMANY' 'BRAZIL' 'NETHERLANDS' 'ITALY' 'POLAND' 'UNK'
 'PORTUGAL' 'BRASIL' 'CZECH' 'AUSTRIA' 'SPAIN' 'ISRAEL']


Column: background_mother
['POMERANIA' 'ITALY' 'GERMANY' 'BRAZIL' 'UNK' 'POLAND' 'NORWAY' 'PORTUGAL'
 'NETHERLANDS' 'FRANCE' 'SPAIN']


Column: age
[55 79 52 74 58 34 78 77 60 64 61 71 38 54 82 62 63 49 31 36 75 44 56 80
 76 48 51 47 35 53 69 59 57 83 92 67 46 66 70 85 41 65 72 14 87 21 94 73
 43 90 89 39 26 40 13 30 68 50 22 81 84 88 86 37 32 45 27 33 20 91 42 17
 16 24]


Column: pesticide
['False' 'True']


Column: gender
['FEMALE' 'MALE']


Column: skin_cancer_history
['True' 'False']


Column: cancer_history
['True' 'False']


Column: has_piped_water
['True' 'False']


Column: has_sewage_system
['True' 'False']


Column: fitspatrick
<IntegerArray>
[3, 1, 2, 4, 5, 6]
Length: 6, dtype: Int64


Column: region
['NECK' 'FOREARM' 'FACE' 'NOSE' 'CHEST' 'THIGH' 'EAR' 'BACK

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Lambda, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
import tensorflow as tf

# Preprocessing
categorical_columns = ['background_father', 'background_mother', 'gender', 'region', 'diagnostic', 
                       'smoke', 'drink', 'pesticide', 'skin_cancer_history', 'cancer_history',
                       'has_piped_water', 'has_sewage_system', 'itch', 'grew', 'hurt', 
                       'changed', 'bleed', 'elevation', 'biopsed']

numeric_columns = ['age', 'fitspatrick', 'diameter_1', 'diameter_2']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ('num', MinMaxScaler(), numeric_columns)
    ])

# Assuming df has been loaded correctly
df_no_nan_preprocessed = preprocessor.fit_transform(df).toarray()  

input_dim = df_no_nan_preprocessed.shape[1]

# Latent dimension
latent_dim = 8  # Adjust as needed

# Custom sampling function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# Define the model architecture
# Encoder
input_layer = Input(shape=(input_dim,))
encoder = Dense(512, activation="relu")(input_layer)
encoder = BatchNormalization()(encoder)
encoder = Dropout(0.3)(encoder)
encoder = Dense(256, activation="relu")(encoder)
encoder = BatchNormalization()(encoder)
encoder = Dropout(0.3)(encoder)

z_mean = Dense(latent_dim)(encoder)
z_log_var = Dense(latent_dim)(encoder)

# Sampling layer using Lambda
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder = Dense(256, activation="relu")(z)
decoder = BatchNormalization()(decoder)
decoder = Dropout(0.3)(decoder)
decoder = Dense(512, activation="relu")(decoder)
decoder = BatchNormalization()(decoder)
decoder = Dense(input_dim, activation="sigmoid")(decoder)

# VAE model
autoencoder = Model(input_layer, decoder)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Callbacks for early stopping, learning rate scheduling, and model checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('best_autoencoder_model.keras', monitor='val_loss', save_best_only=True)

# Train the model
autoencoder.fit(df_no_nan_preprocessed, df_no_nan_preprocessed, 
                epochs=300,
                batch_size=32, 
                shuffle=True, 
                validation_split=0.2, 
                callbacks=[early_stopping, reduce_lr, model_checkpoint])

# Load the model weights
autoencoder.load_weights("best_autoencoder_model.keras")

# Now the model with loaded weights is ready for evaluation or further processing

Epoch 1/300
37/37 [==============================] - 3s 23ms/step - loss: 0.2331 - val_loss: 0.1974 - lr: 0.0010
Epoch 2/300
37/37 [==============================] - 0s 13ms/step - loss: 0.1451 - val_loss: 0.1203 - lr: 0.0010
Epoch 3/300
37/37 [==============================] - 0s 11ms/step - loss: 0.0759 - val_loss: 0.0858 - lr: 0.0010
Epoch 4/300
37/37 [==============================] - 0s 12ms/step - loss: 0.0548 - val_loss: 0.0773 - lr: 0.0010
Epoch 5/300
37/37 [==============================] - 0s 8ms/step - loss: 0.0485 - val_loss: 0.0696 - lr: 0.0010
Epoch 6/300
37/37 [==============================] - 0s 10ms/step - loss: 0.0461 - val_loss: 0.0613 - lr: 0.0010
Epoch 7/300
37/37 [==============================] - 0s 10ms/step - loss: 0.0441 - val_loss: 0.0535 - lr: 0.0010
Epoch 8/300
37/37 [==============================] - 0s 12ms/step - loss: 0.0428 - val_loss: 0.0478 - lr: 0.0010
Epoch 9/300
37/37 [==============================] - 0s 10ms/step - loss: 0.0408 - val_loss: 0.04

In [6]:
# Imputation function

df = dfcopy.copy()


In [7]:
import numpy as npbest_model
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Model
from keras.layers import Input, Dense


def fill_nan_with_autoencoder(df, preprocessor, model, categorical_columns, numeric_columns):
    # Temporarily fill NaN values with 0 and preprocess the data
    df_preprocessed = preprocessor.transform(df.fillna(0))
    
    # Ensure the preprocessed data is numeric and in float32 format
    df_preprocessed = df_preprocessed.astype(np.float32)
    
    # Use the autoencoder to predict the missing values
    reconstructed = model.predict(df_preprocessed)
    
    # Manually inverse-transform the preprocessed data:
    
    # 1. Inverse-transform the numeric columns using MinMaxScaler
    numeric_scaler = preprocessor.named_transformers_['num']
    numeric_indices = [df.columns.get_loc(col) for col in numeric_columns]
    reconstructed_numeric = numeric_scaler.inverse_transform(reconstructed[:, numeric_indices])
    
    # Round the reconstructed numeric values to ensure compatibility with integer columns
    reconstructed_numeric = np.round(reconstructed_numeric).astype(int)
    
    # 2. Inverse-transform the categorical columns using OneHotEncoder
    cat_encoder = preprocessor.named_transformers_['cat']
    
    # Calculate the total number of one-hot encoded columns
    total_onehot_columns = sum(len(categories) for categories in cat_encoder.categories_)
    
    # Extract the one-hot encoded section from reconstructed data
    start_idx = reconstructed.shape[1] - total_onehot_columns
    reconstructed_categorical = cat_encoder.inverse_transform(reconstructed[:, start_idx:])
    
    # Reconstruct the full dataframe:
    df_filled = df.copy()
    
    # Fill the numeric columns with the inverse-transformed values
    for i, col in enumerate(numeric_columns):
        na_indices = df[col].isna()  # Find the positions where NaN exists
        df_filled.loc[na_indices, col] = reconstructed_numeric[na_indices, i]  # Assign only those values
    
    # Fill the categorical columns with inverse-transformed values
    for i, col in enumerate(categorical_columns):
        na_indices = df[col].isna()  # Find the positions where NaN exists
        df_filled.loc[na_indices, col] = reconstructed_categorical[na_indices, i]  # Assign only those values
    
    return df_filled

# Apply to rows with NaN
df_filled = fill_nan_with_autoencoder(df_with_na, preprocessor, autoencoder, categorical_columns, numeric_columns)

print(df_filled)


26/26 [==============================] - 0s 2ms/step
      smoke  drink background_father background_mother  age pesticide  gender  \
0     False  False               UNK          PORTUGAL    8      True  FEMALE   
2      True   True               UNK          PORTUGAL   77      True  FEMALE   
3      True   True               UNK          PORTUGAL   75      True  FEMALE   
5     False   True               UNK          PORTUGAL   53      True    MALE   
8      True  False               UNK          PORTUGAL   68     False  FEMALE   
...     ...    ...               ...               ...  ...       ...     ...   
2288   True  False            BRAZIL          PORTUGAL   81      True  FEMALE   
2290  False   True               UNK          PORTUGAL   35      True  FEMALE   
2293   True   True               UNK          PORTUGAL   73      True  FEMALE   
2295   True  False               UNK          PORTUGAL   74      True  FEMALE   
2297  False  False               UNK          PORTUGAL  

In [8]:
for column in df_filled.columns:
    print(f"Column: {column}")
    print(df_filled[column].unique())
    print("\n")

Column: smoke
['False' 'True']


Column: drink
['False' 'True']


Column: background_father
['UNK' 'ISRAEL' 'BRAZIL' 'ITALY' 'POMERANIA' 'GERMANY' 'AUSTRIA' 'SPAIN']


Column: background_mother
['PORTUGAL' 'POMERANIA' 'ITALY' 'GERMANY']


Column: age
[ 8 77 75 53 68 45  9 62 67 66 52 78 76 63 58 17 26 47 74 87 41 22 60 65
 80 33 56 64 36 50 59 40 54 73 44 10 86 79 61 57 69 39 72 43 46 55 35 38
 71 48 32 16 21 82 29 81 31 24 42 20 83 23 84 88 34 28 70 18 49 37 89 51
 12 14 27 25  6 13 30 92]


Column: pesticide
['True' 'False']


Column: gender
['FEMALE' 'MALE']


Column: skin_cancer_history
['True' 'False']


Column: cancer_history
['False' 'True']


Column: has_piped_water
['False' 'True']


Column: has_sewage_system
['True' 'False']


Column: fitspatrick
<IntegerArray>
[1, 2, 4, 3]
Length: 4, dtype: Int64


Column: region
['ARM' 'FACE' 'HAND' 'CHEST' 'FOREARM' 'NECK' 'SCALP' 'EAR' 'BACK' 'NOSE'
 'ABDOMEN' 'THIGH' 'FOOT' 'LIP']


Column: diameter_1
<IntegerArray>
[1, 3, 2, 7, 4, 5, 6,

In [9]:
import numpy as np

for index, row in df.iterrows():
    for col in df.columns:
        if pd.isna(row[col]):
            df.at[index, col] = df_filled.at[index, col]

# Print unique values of each column
for column in df.columns:
    print(f"Column: {column}")
    print(df[column].unique())
    print("\n")

Column: patient_id
['PAT_1516' 'PAT_46' 'PAT_1545' ... 'PAT_491' 'PAT_1343' 'PAT_1714']


Column: lesion_id
[1765  881 1867 ...  934 3156 3189]


Column: smoke
['False' 'True']


Column: drink
['False' 'True']


Column: background_father
['UNK' 'POMERANIA' 'GERMANY' 'BRAZIL' 'NETHERLANDS' 'ISRAEL' 'ITALY'
 'POLAND' 'PORTUGAL' 'BRASIL' 'CZECH' 'AUSTRIA' 'SPAIN']


Column: background_mother
['PORTUGAL' 'POMERANIA' 'ITALY' 'GERMANY' 'BRAZIL' 'UNK' 'POLAND' 'NORWAY'
 'NETHERLANDS' 'FRANCE' 'SPAIN']


Column: age
[ 8 55 77 75 79 53 52 74 68 58 45 34  9 78 60 64 61 62 67 71 66 38 54 82
 63 49 76 31 36 17 44 56 26 47 80 48 51 35 87 41 22 69 59 57 65 83 92 33
 46 50 70 85 40 73 10 86 72 14 21 39 94 43 90 89 13 30 81 84 88 37 32 16
 29 24 42 20 23 28 27 91 18 12 25  6]


Column: pesticide
['True' 'False']


Column: gender
['FEMALE' 'MALE']


Column: skin_cancer_history
['True' 'False']


Column: cancer_history
['False' 'True']


Column: has_piped_water
['False' 'True']


Column: has_sewage_syst

In [10]:
tempdf = df.copy()
df = df.drop(columns=['patient_id', 'lesion_id', 'img_id'])
for column in df.columns:
    print(f"Column: {column}")
    print(df[column].unique())
    print("\n")

Column: smoke
['False' 'True']


Column: drink
['False' 'True']


Column: background_father
['UNK' 'POMERANIA' 'GERMANY' 'BRAZIL' 'NETHERLANDS' 'ISRAEL' 'ITALY'
 'POLAND' 'PORTUGAL' 'BRASIL' 'CZECH' 'AUSTRIA' 'SPAIN']


Column: background_mother
['PORTUGAL' 'POMERANIA' 'ITALY' 'GERMANY' 'BRAZIL' 'UNK' 'POLAND' 'NORWAY'
 'NETHERLANDS' 'FRANCE' 'SPAIN']


Column: age
[ 8 55 77 75 79 53 52 74 68 58 45 34  9 78 60 64 61 62 67 71 66 38 54 82
 63 49 76 31 36 17 44 56 26 47 80 48 51 35 87 41 22 69 59 57 65 83 92 33
 46 50 70 85 40 73 10 86 72 14 21 39 94 43 90 89 13 30 81 84 88 37 32 16
 29 24 42 20 23 28 27 91 18 12 25  6]


Column: pesticide
['True' 'False']


Column: gender
['FEMALE' 'MALE']


Column: skin_cancer_history
['True' 'False']


Column: cancer_history
['False' 'True']


Column: has_piped_water
['False' 'True']


Column: has_sewage_system
['True' 'False']


Column: fitspatrick
<IntegerArray>
[1, 3, 2, 4, 5, 6]
Length: 6, dtype: Int64


Column: region
['ARM' 'NECK' 'FACE' 'HAND' '

In [13]:
# tempdf.to_csv("filledValues.csv")